# 車を検出し、その周りに境界ボックスを配置する

このノートブックでは、YOLOモデルによって検出された車の周りに境界ボックスを描画する方法を学びます。

まず、YOLOパッケージをインポートします。

In [ ]:
# もしこのLabのために設計されたWorkbenchイメージを使用していない場合、必要なパッケージをインストールするために次の行のコメントを外して実行できます。
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

from ultralytics import YOLO
from PIL import Image

In [ ]:
# 物体検出にYOLOv8mモデルを利用します。

model = YOLO("yolov8m.pt")

In [ ]:
# テスト画像に対する予測結果を表示します。

img = "images/carImage0.jpg"
results = model.predict(img)

In [ ]:
# YOLOでは1つの画像だけでなく、画像の配列を送信することができ、結果の配列を取得できます。
# 今回は1つの画像のみを送信したため、結果の配列から最初（そして唯一）の要素を取得する必要があります。

result = results[0]

In [ ]:
# 検出されたボックスの数を確認します。

len(result.boxes)

In [ ]:
# ボックスを分析します。

box = result.boxes[0]
print("オブジェクトの種類:", box.cls)
print("座標:", box.xyxy)
print("確率:", box.conf)

In [ ]:
# テンソルから実際の値を展開します。

cords = box.xyxy[0].tolist()
class_id = box.cls[0].item()
conf = box.conf[0].item()
print("オブジェクトの種類:", class_id)
print("座標:", cords)
print("確率:", conf)

In [ ]:
# YOLOモデルが訓練されたデータセットであるCOCOには、検出するオブジェクトがクラスごとに整理されています。これは「Object type」フィールドで得られる情報です。
# YOLOv8の結果オブジェクトには、これらのクラスのための「names」プロパティも含まれています。

print(result.names)

クラス番号 '2' が 'car' オブジェクトに対応していることがわかります。したがって、結果の境界ボックスは検出された「車」に対応しています。
それでは、画像にボックスを描画しましょう。


In [ ]:
# 最初に、座標をリストに入れて丸めます。
# 次に、result.names ディクショナリを使用して、検出されたオブジェクトのクラス名をIDから取得します。

cords = box.xyxy[0].tolist()
cords = [round(x) for x in cords]
class_id = result.names[box.cls[0].item()]
conf = round(box.conf[0].item(), 2)
print("オブジェクトの種類:", class_id)
print("座標:", cords)
print("確率:", conf)

In [ ]:
# すべてのボックスをループして情報を抽出しましょう。

for box in result.boxes:
  class_id = result.names[box.cls[0].item()]
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2)
  print("オブジェクトの種類:", class_id)
  print("座標:", cords)
  print("確率:", conf)
  print("---")

In [ ]:
# 画像上に、ボックス、クラス名、および確率（モデルが検出についてどれだけ確信しているか）を描画します。

Image.fromarray(result.plot()[:,:,::-1])

前回のノートブックでテストした複数の車両が含まれる画像（carImage4.jpg）に戻り、YOLOが画像内のすべての「車」を正しく識別できるかを確認しましょう。

In [ ]:
# 前のセルで使用したコードと同じですが、1つのセルで実行します。

results = model.predict("images/carImage4.jpg")

result = results[0]

for box in result.boxes:
  class_id = result.names[box.cls[0].item()]
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2)
  print("Object type:", class_id)
  print("Coordinates:", cords)
  print("Probability:", conf)
  print("---")

Image.fromarray(result.plot()[:,:,::-1])

YOLOモデルは画像の「奥にある」いくつかの車を見逃していることがわかります。しかし全体として、このモデルはこの画像内の複数の車を正しく識別することができました。さらに重要なことは、識別された車両が「境界ボックス」で囲まれていることがわかるという点です！

これで、YOLOモデルが認識した車に境界ボックスを配置できるようになったので、モデルを再トレーニングして車の「衝突」を識別できるようにすることができます。